In [11]:
from files.utils.constants import *
from files.utils.utility_functions import *
from files.utils.dataset_creator import *
from files.utils.dataset_3d_helper import *
from files.classes.neural_models import NeuralNetwork


In [12]:
plane, gt = create_plane(of=0, noise_std=0)
plot3d(plane)


In [13]:
ae = NeuralNetwork(neurons=[3, 2, 3], activation=lambda x: x)
ae.fit(data=plane, epochs=300, print_training=True)


Epoch 300/300: [==============================] (100%) ETA: 0:00:00.040512 Mean Loss: 0.00311


In [14]:
predicted_plane = ae.predict(plane)

plot3d(predicted_plane)


In [15]:
weights = []
for n, v in ae.named_parameters():
    weights.append(v.detach().numpy())
len(weights)


4

In [16]:
weights


[array([[ 0.918101  , -0.13523126, -0.4898178 ],
        [-0.12385109,  0.8030569 , -0.44092953]], dtype=float32),
 array([ 0.01071012, -0.0025083 ], dtype=float32),
 array([[ 0.8675651 , -0.09295646],
        [-0.08188179,  0.98579705],
        [-0.39284712, -0.44642013]], dtype=float32),
 array([-0.00953187,  0.00336198,  0.00309118], dtype=float32)]

In [17]:
w_5 = weights[2][0]
w_6 = weights[2][1]
w_7 = weights[2][2]

w_35, w_45 = w_5
w_36, w_46 = w_6
w_37, w_47 = w_7


In [18]:
b_5, b_6, b_7 = weights[-1]


In [19]:
# 00 --> 35
# 01 --> 36
# 02 --> 37

# 10 --> 45
# 11 --> 46
# 12 --> 47

# z = a_d*x +b_d*y + c_d
# a / -c
a_c = (w_37 * w_46 - w_47 * w_36) / (w_35 * w_46 - w_45 * w_36)
# b / -c
b_c = (w_47 * w_35 - w_37 * w_45) / (w_35 * w_46 - w_45 * w_36)
# d / -c
d_c = b_7 - a_c*b_5 - b_c*b_6

a_c, b_c, d_c


(-0.5000064, -0.5000005, 6.178394e-06)

In [2]:
import sympy as sy
sy.init_printing()

syms = w_35, w_36, w_45, w_46, w_37, w_47, h_3_1, h_3_2, h_3_3, h_4_1, h_4_2, h_4_3, b_5, b_6, b_7, x_1, x_2, x_3, y_1, y_2, y_3, z_1, z_2, z_3 = sy.symbols(
    "w_35 w_36 w_45 w_46 w_37 w_47 h_36 h_37 h_03 h_46 h_47 h_13 b_5 b_6 b_7 x_1 x_2 x_3 y_1 y_2 y_3 z_1 z_2 z_3")

# a = -(-y_2*z_1 + y_3*z_1 + y_1*z_2 - y_3*z_2 - y_1*z_3 + y_2*z_3)
# b = -(x_2*z_1 - x_3*z_1 -x_1*z_2 + x_3*z_2 + x_1*z_3 - x_2*z_3)
# c = -x_2*y_1 + x_3*y_1 + x_1*y_2 - x_3*y_2 - x_1*y_3 + x_2*y_3
# d = x_3*y_2*z_1 - x_2*y_3*z_1 - x_3*y_1*z_2 + x_1*y_3*z_2 + x_2*y_1*z_3 - x_1*y_2*z_3

# ax + by + cz + d = 0


In [3]:
def compute_a_b_c_d(x_1, x_2, x_3, y_1, y_2, y_3, z_1, z_2, z_3):

    M = sy.Matrix([[y_2-y_1, z_2-z_1], [y_3-y_1, z_3-z_1]])
    a = sy.simplify(M.det())

    M = sy.Matrix([[x_2-x_1, z_2-z_1], [x_3-x_1, z_3-z_1]])
    b = sy.simplify(-M.det())

    M = sy.Matrix([[x_2-x_1, y_2-y_1], [x_3-x_1, y_3-y_1]])
    c = sy.simplify(M.det())

    d = sy.simplify(-a*x_1 - b*y_1 - c*z_1)

    return a, b, c, d


a, b, c, d = compute_a_b_c_d(x_1, x_2, x_3, y_1, y_2, y_3, z_1, z_2, z_3)

print(sy.latex(a) + "\n")
print(sy.latex(b) + "\n")
print(sy.latex(c) + "\n")
print(sy.latex(d) + "\n")


y_{1} z_{2} - y_{1} z_{3} - y_{2} z_{1} + y_{2} z_{3} + y_{3} z_{1} - y_{3} z_{2}

- x_{1} z_{2} + x_{1} z_{3} + x_{2} z_{1} - x_{2} z_{3} - x_{3} z_{1} + x_{3} z_{2}

x_{1} y_{2} - x_{1} y_{3} - x_{2} y_{1} + x_{2} y_{3} + x_{3} y_{1} - x_{3} y_{2}



In [14]:
x_1 = w_35*h_3_1+w_45*h_4_1+b_5
x_2 = w_35*h_3_2+w_45*h_4_2+b_5
x_3 = w_35*h_3_3+w_45*h_4_3+b_5

y_1 = w_36*h_3_1+w_46*h_4_1+b_6
y_2 = w_36*h_3_2+w_46*h_4_2+b_6
y_3 = w_36*h_3_3+w_46*h_4_3+b_6

z_1 = w_37*h_3_1+w_47*h_4_1+b_7
z_2 = w_37*h_3_2+w_47*h_4_2+b_7
z_3 = w_37*h_3_3+w_47*h_4_3+b_7


In [ ]:
a, b, c, d = compute_a_b_c_d(x_1, x_2, x_3, y_1, y_2, y_3, z_1, z_2, z_3)

print(sy.latex(a) + "\n")
print(sy.latex(b) + "\n")
print(sy.latex(c) + "\n")
print(sy.latex(d) + "\n")


In [18]:
sy.latex(sy.simplify(d/(-c), syms=syms))


'\\frac{b_{5} w_{36} w_{47} - b_{5} w_{37} w_{46} - b_{6} w_{35} w_{47} + b_{6} w_{37} w_{45} + b_{7} w_{35} w_{46} - b_{7} w_{36} w_{45}}{w_{35} w_{46} - w_{36} w_{45}}'

In [19]:
sy.latex(sy.simplify(a/(-c), syms=syms))


'\\frac{- w_{36} w_{47} + w_{37} w_{46}}{w_{35} w_{46} - w_{36} w_{45}}'

In [20]:
sy.latex(sy.simplify(b/(-c), syms=syms))


'\\frac{w_{35} w_{47} - w_{37} w_{45}}{w_{35} w_{46} - w_{36} w_{45}}'

In [1]:
import sympy as sy

# function to compute coefficients of the plane


def compute_a_b_c_d(x_1, x_2, x_3, y_1, y_2, y_3, z_1, z_2, z_3):
    M = sy.Matrix([[y_2-y_1, z_2-z_1], [y_3-y_1, z_3-z_1]])
    a = sy.simplify(M.det())

    M = sy.Matrix([[x_2-x_1, z_2-z_1], [x_3-x_1, z_3-z_1]])
    b = sy.simplify(-M.det())

    M = sy.Matrix([[x_2-x_1, y_2-y_1], [x_3-x_1, y_3-y_1]])
    c = sy.simplify(M.det())

    d = sy.simplify(-a*x_1 - b*y_1 - c*z_1)

    return a, b, c, d


# initialize printing system
sy.init_printing()
# define symbols for calculations
syms = w_35, w_36, w_45, w_46, \
        w_37, w_47, h_3_1, h_3_2, \
        h_3_3, h_4_1, h_4_2, h_4_3, \
        b_5, b_6, b_7, x_1, \
        x_2, x_3, y_1, y_2, \
        y_3, z_1, z_2, z_3 = sy.symbols(
                                "w_35 w_36 w_45 w_46 " +
                                "w_37 w_47 h_3_1 h_3_2 " +
                                "h_3_3 h_4_1 h_4_2 h_4_3 " +
                                "b_5 b_6 b_7 x_1 " +
                                "x_2 x_3 y_1 y_2 " +
                                "y_3 z_1 z_2 z_3 ")


a, b, c, d = compute_a_b_c_d(x_1, x_2, x_3, y_1, y_2, y_3, z_1, z_2, z_3)

print("a: \n" + sy.latex(a) + "\n")
print("b: \n" + sy.latex(b) + "\n")
print("c: \n" + sy.latex(c) + "\n")
print("d: \n" + sy.latex(d) + "\n")

# define coordinates from network's output
x_1 = w_35*h_3_1+w_45*h_4_1+b_5
x_2 = w_35*h_3_2+w_45*h_4_2+b_5
x_3 = w_35*h_3_3+w_45*h_4_3+b_5

y_1 = w_36*h_3_1+w_46*h_4_1+b_6
y_2 = w_36*h_3_2+w_46*h_4_2+b_6
y_3 = w_36*h_3_3+w_46*h_4_3+b_6

z_1 = w_37*h_3_1+w_47*h_4_1+b_7
z_2 = w_37*h_3_2+w_47*h_4_2+b_7
z_3 = w_37*h_3_3+w_47*h_4_3+b_7

a, b, c, d = compute_a_b_c_d(x_1, x_2, x_3, y_1, y_2, y_3, z_1, z_2, z_3)

print("a: \n" + sy.latex(a) + "\n")
print("b: \n" + sy.latex(b) + "\n")
print("c: \n" + sy.latex(c) + "\n")
print("d: \n" + sy.latex(d) + "\n")

print("-(a/c): \n" + sy.latex(sy.simplify(-(a/c), syms=syms)) + "\n")
print("-(b/c): \n" + sy.latex(sy.simplify(-(b/c), syms=syms)) + "\n")
print("-(d/c): \n" + sy.latex(sy.simplify(-(d/c), syms=syms)) + "\n")


a: 
y_{1} z_{2} - y_{1} z_{3} - y_{2} z_{1} + y_{2} z_{3} + y_{3} z_{1} - y_{3} z_{2}

b: 
- x_{1} z_{2} + x_{1} z_{3} + x_{2} z_{1} - x_{2} z_{3} - x_{3} z_{1} + x_{3} z_{2}

c: 
x_{1} y_{2} - x_{1} y_{3} - x_{2} y_{1} + x_{2} y_{3} + x_{3} y_{1} - x_{3} y_{2}

d: 
- x_{1} y_{2} z_{3} + x_{1} y_{3} z_{2} + x_{2} y_{1} z_{3} - x_{2} y_{3} z_{1} - x_{3} y_{1} z_{2} + x_{3} y_{2} z_{1}

a: 
h_{3 1} h_{4 2} w_{36} w_{47} - h_{3 1} h_{4 2} w_{37} w_{46} - h_{3 1} h_{4 3} w_{36} w_{47} + h_{3 1} h_{4 3} w_{37} w_{46} - h_{3 2} h_{4 1} w_{36} w_{47} + h_{3 2} h_{4 1} w_{37} w_{46} + h_{3 2} h_{4 3} w_{36} w_{47} - h_{3 2} h_{4 3} w_{37} w_{46} + h_{3 3} h_{4 1} w_{36} w_{47} - h_{3 3} h_{4 1} w_{37} w_{46} - h_{3 3} h_{4 2} w_{36} w_{47} + h_{3 3} h_{4 2} w_{37} w_{46}

b: 
- h_{3 1} h_{4 2} w_{35} w_{47} + h_{3 1} h_{4 2} w_{37} w_{45} + h_{3 1} h_{4 3} w_{35} w_{47} - h_{3 1} h_{4 3} w_{37} w_{45} + h_{3 2} h_{4 1} w_{35} w_{47} - h_{3 2} h_{4 1} w_{37} w_{45} - h_{3 2} h_{4 3} w_{35} w_{4